In [1]:
import pandas as pd
import requests
import json
import time

from config import api_key

In [2]:
# pull in the url's scraped from YouTube
url_file =  open('url_list.txt', 'r') 
# append each url to a list of url's
url_list = []
for line in url_file:
    stripped_line = line.strip()
    url_list.append(stripped_line)

In [3]:
# append the video identifier from each url to a list of video id's
video_id_list = []
for i in url_list:
    video_id_list.append(i.split('=', maxsplit=1)[1])

In [4]:
# create a list to store the dict values of each api pull
video_metadata = []

In [7]:
# assign count
count = 0

# start time to calculate the time it takes for the script to run
start_time = time.time()

for vidId in video_id_list:
    count += 1
    try:
        # pull all the necessary json data from the API
        # video data
        vid_snip = f'https://www.googleapis.com/youtube/v3/videos?part=snippet&id={vidId}&key={api_key}'
        vid_snip_req = requests.get(vid_snip).json()
        ## get the video channel id to gather channel data in channel api's
        vid_chan = vid_snip_req['items'][0]['snippet']['channelId']
        vid_stat = f'https://www.googleapis.com/youtube/v3/videos?part=statistics&id={vidId}&key={api_key}'
        vid_stat_req = requests.get(vid_stat).json()
        vid_cont = f'https://www.googleapis.com/youtube/v3/videos?part=contentDetails&id={vidId}&key={api_key}'
        vid_cont_req = requests.get(vid_cont).json()
        # channel data
        chan_snip = f'https://www.googleapis.com/youtube/v3/channels?part=snippet&id={vid_chan}&key={api_key}'
        chan_snip_req = requests.get(chan_snip).json()
        chan_stat = f'https://www.googleapis.com/youtube/v3/channels?part=statistics&id={vid_chan}&key={api_key}'
        chan_stat_req = requests.get(chan_stat).json()

        # locate and assign desired embeded values to an identifying variable name
        try: vidId = vid_snip_req['items'][0]['id']
        except: vidId = -1

        try: date = vid_snip_req['items'][0]['snippet']['publishedAt']
        except: date = -1

        try: title = vid_snip_req['items'][0]['snippet']['title']
        except: title = -1

        try: description = vid_snip_req['items'][0]['snippet']['description']
        except: description = -1

        try: channel = vid_snip_req['items'][0]['snippet']['channelTitle']
        except: channel = -1

        try: viewCount = vid_stat_req['items'][0]['statistics']['viewCount']
        except: viewCount = -1

        try: likeCount = vid_stat_req['items'][0]['statistics']['likeCount']
        except: likeCount = -1

        try: dislikeCount = vid_stat_req['items'][0]['statistics']['dislikeCount']
        except: dislikeCount = -1

        try: commentCount = vid_stat_req['items'][0]['statistics']['commentCount']
        except: commentCount = -1

        try: vidLength = vid_cont_req['items'][0]['contentDetails']['duration']
        except: vidLength = -1

        try: country = chan_snip_req['items'][0]['snippet']['country']
        except: country = -1

        try: totViews = chan_stat_req['items'][0]['statistics']['viewCount']
        except: totViews = -1

        try: totSubscribers = chan_stat_req['items'][0]['statistics']['subscriberCount']
        except: totSubscribers = -1

        try: totVideos = chan_stat_req['items'][0]['statistics']['videoCount']
        except: totVideos = -1

        # append all variables to a list of dictionaries called 'video_metadata' from above
        video_metadata.append({"vidId": vidId,
                               "date": date,
                               "title": title, 
                               "views": viewCount,
                               "likes": likeCount,
                               "dislikes": dislikeCount,
                               "commentCount":commentCount,
                               "vidLength": vidLength,
                               "description": description,
                               "channel": channel,
                               "country": country,
                               "totChanViews": totViews,
                               "totSubscribers": totSubscribers,
                               "totChanVideos": totVideos})

        print('Count:', count, 'Title:', title)
    except:
        print('Count:', count, "...DID NOT PULL...")

print("\n\nDONE! This script took", round((time.time() - start_time)/60, 2), "minutes to run.")

Count: 1 Title: Indian Parents Give Me “The Talk.” It’s Not What I Expected!
Count: 2 Title: Halloween As An Adult vs A Kid
Count: 3 Title: The birth of my son!
Count: 4 Title: I Filmed My Dog Whenever He Cried Or Was Nasty For 24 Hours
Count: 5 Title: victorious as zodiac signs
Count: 6 Title: We visited the world's SCARIEST haunted house...
Count: 7 Title: Do You Hear "Yanny" or "Laurel"? (SOLVED with SCIENCE)
Count: 8 Title: How Girls Get Ready...
Count: 9 Title: KID vs ADULT - Extreme Acro Gymnastics Competition
Count: 10 Title: TELLING MY FRIENDS I'M PREGNANT WITH TWINS!
Count: 11 Title: 13 Types of Students Studying for Exams
Count: 12 Title: BTR Worldwide 2020
Count: 13 Title: I Flew Emirates First Class and It Was Insane! | Noah Schnapp
Count: 14 Title: Chopping Off My Hair
Count: 15 Title: 3 Marker Plushie Challenge ~ Fun DIY Doodle Plushie ~ Jacy and Kacy
Count: 16 Title: Five Nights at Freddy's: Security Breach - State of Play Oct 2021 Trailer | PS5, PS4
Count: 17 Title: The

In [8]:
df = pd.DataFrame(video_metadata) 
df.head()

,vidId,date,title,views,likes,dislikes,commentCount,vidLength,description,channel,country,totChanViews,totSubscribers,totChanVideos
0,s2ZTZrghxvg,2021-11-04T16:00:12Z,Indian Parents Give Me “The Talk.” It’s Not Wh...,125462,14308,109,639,PT3M40S,You’ve heard of the Kama Sutra. But allow me t...,Lilly Singh,-1,3489529500,14700000,836
1,ce4V_PYLhEo,2018-10-22T21:57:57Z,Halloween As An Adult vs A Kid,8157273,163837,3156,5633,PT2M31S,Remember when Halloween was an excuse to go bu...,Lilly Singh,-1,3489529500,14700000,836
2,yK6OiJ2sn_o,2018-12-14T02:02:30Z,The birth of my son!,31133838,1425411,10718,196439,PT14M11S,I love him so much. He is absolutely perfect.\...,Colleen Ballinger,-1,1818054569,8670000,1062
3,xf0Mi3kWKhY,2019-01-24T06:02:13Z,I Filmed My Dog Whenever He Cried Or Was Nasty...,19022064,695716,18977,70981,PT13M14S,Please subscribe to my channel and my vlog cha...,JennaMarbles,-1,1723095428,20000000,250
4,ETvPH2pFzVs,2021-03-14T09:41:11Z,victorious as zodiac signs,525335,16969,174,895,PT5M27S,• [watch in 1080p for a better quality] •\n\n...,elveyn brown,-1,1151416,2180,11


#### Export the dataframe to a csv 

In [9]:
##  make a new csv
# df.to_csv('youtube_metadata.csv', index=False)
## append to an existing csv
df.to_csv('youtube_metadata.csv', mode='a', header=False, index=False)